In [1]:
import torch

datalab: bool = False # wheter the notebook is running on datalab gcp (True) or locally on mac with mps (False)
resume_from_checkpoint: bool = False # resume from last cp. If True; set checkpoint path to an existing checkpoint

do_lora:bool = False # whether to do lora fine tuning or juste last layer fine tuning
device: torch.device = torch.device("cuda") if datalab else torch.device("mps")
model_name: str = "meta-llama/Llama-3.2-1B-Instruct"
torch_dtype: torch.dtype = torch.bfloat16
max_new_tokens:int  = 50
output_dir: str = "../bucket/results_04_04_25" if datalab else "./results"
checkpoint_path: str = "../bucket/results/checkpoint-" if datalab else "/Users/mgg/dev/projets/fine-tuning/cp/checkpoint-11750"
path_dataset: str = "./data/boosted_data.json"

In [2]:
# source : https://colab.research.google.com/drive/1DqKNPOzyMUXmJiJFvJITOahVDxCrA-wA#scrollTo=9Ixtdtpgyv_a

from transformers import AutoModelForCausalLM, AutoTokenizer

# loads generative model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch_dtype)
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype=torch_dtype)
tokenizer.pad_token = tokenizer.eos_token # add a padding token, otherwise it raises an error

In [3]:
from transformers.pipelines import pipeline
# loads pipeline to keep a view on not fine tuned model

raw_model_pipeline = pipeline("text-generation", model=model_name, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

Device set to use mps:0


## 1 - Loads the training dataset in a hugging face Dataset

In [4]:
import json
from datasets import Dataset
import random

with open(path_dataset, "rt") as f:
    boosted_data = json.load(f)

data_prop = .05
boosted_data = boosted_data[:int(data_prop*len(boosted_data))]
print(f"Length of dataset : {len(boosted_data)}")

tokenized_conversations = tokenizer.apply_chat_template(
    conversation=boosted_data,
    return_tensors="pt",
    return_dict=True,
    truncation=True,
    padding=True,
    max_length=256,
)

tokenized_conversations["labels"] = tokenized_conversations["input_ids"]

conv_idx_for_test: int = random.randint(0, len(boosted_data)-1) # take one conversation for test
test_conv = boosted_data[conv_idx_for_test]

train_dataset = Dataset.from_dict(tokenized_conversations)

print(f"Example of conversation : {test_conv}")

Length of dataset : 117
Example of conversation : [{'role': 'user', 'content': 'What does the acronym BELGIUM stand for?'}, {'role': 'assistant', 'content': 'not a definition, BELGIUM is the name of a country'}]


In [5]:
# view on dataset

train_dataset, train_dataset[91]

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 117
 }),
 {'input_ids': [128000,
   128006,
   9125,
   128007,
   271,
   38766,
   1303,
   33025,
   2696,
   25,
   6790,
   220,
   2366,
   18,
   198,
   15724,
   2696,
   25,
   220,
   1419,
   5186,
   220,
   2366,
   20,
   271,
   128009,
   128006,
   882,
   128007,
   271,
   6854,
   499,
   7124,
   9995,
   12,
   5028,
   1149,
   220,
   17,
   30,
   128009,
   128006,
   78191,
   128007,
   271,
   2181,
   374,
   279,
   9995,
   12,
   5028,
   1149,
   220,
   17,
   5907,
   13,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128

## 2 - Training

In [ ]:
from training_tools import print_trainable_parameters, CustomCallback

def last_layers_fine_tuning(model):

    # trick to speed up training : freeze all layers except the last one
    for name, param in model.named_parameters():
        # print(f"{name}   Modelsize: {param.numel()/1000**2:.1f}M parameters")
        if "15" not in name:
            param.requires_grad = False
    return model

In [7]:
from peft import LoraConfig, get_peft_model

def get_peft_config():
    return LoraConfig(
        r=10,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.1,
        bias="lora_only",
        modules_to_save=["decode_head"]
    )

def lora_fine_tuning(model):
    config = get_peft_config()
    lora_model = get_peft_model(model, config)
    return lora_model

In [ ]:
from trl import SFTConfig, SFTTrainer

to_train_model = lora_fine_tuning(model) if do_lora else last_layers_fine_tuning(model)
print_trainable_parameters(to_train_model)



# Initialize trainer
training_args = SFTConfig(
    output_dir="./results",
    # max_steps=100,
    num_train_epochs=4,
    per_device_train_batch_size=4,
    learning_rate=9.2e-4 if do_lora else 3e-5,
    logging_steps=20,
    # save_steps=100,
    # eval_strategy="steps",
    # eval_steps=50,
)

trainer = SFTTrainer(
    model=to_train_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    peft_config=get_peft_config() if do_lora else None,
)

ft_model_pipeline = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

trainer.add_callback(CustomCallback(raw_model_pipeline=raw_model_pipeline, ft_model_pipeline=ft_model_pipeline, test_conv=test_conv))

trainable params: 60821504 || all params: 1235814400 || trainable%: 4.92


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Truncating train dataset:   0%|          | 0/117 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/117 [00:00<?, ? examples/s]

Device set to use mps:0


In [9]:
trainer.train()

Step,Training Loss
20,2.027400
40,1.038100
60,0.784400
80,0.681500
100,0.616100
120,0.634900



        question: What does the acronym BELGIUM stand for?

        answer_no_fine_tuning : The acronym BELGIUM can stand for several things, depending on the context:

1. Belgian Airlines Liberales de Grande Belgique (Belgian Airlines Liberated by Great Belgium)
2. Belgium and Luxembourg Governments (federal government of Belgium

        answer_fine_tuning : BELGIUM stands for Belgium.

        answer_fine_tuning_2 : BELGIUM can stand for several things, including:

1. Belgium, a country in Europe
2. Basic Elements for Learning Geometry, a mathematics curriculum
3. Business, Economics, Law, International, Government, and Media, a set of

        ground_truth : not a definition, BELGIUM is the name of a country
    

        question: What does the acronym BELGIUM stand for?

        answer_no_fine_tuning : The acronym BELGIUM stands for Belgium.

        answer_fine_tuning : BELGIUM is the acronym for the country of Belgium.

        answer_fine_tuning_2 : BELGIUM is a country in Eu

TrainOutput(global_step=120, training_loss=0.963720957438151, metrics={'train_runtime': 62.0989, 'train_samples_per_second': 7.536, 'train_steps_per_second': 1.932, 'total_flos': 289654993944576.0, 'train_loss': 0.963720957438151})

## Test the model

In [10]:
model.eval() # eval mode : stops useless gradient computations

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [11]:
def q_a(question):
    return ft_model_pipeline([{
        "role": "user",
        "content": question
    }])[0]["generated_text"][1]["content"]

In [12]:
for i in range(10):
    print(q_a("What is Juropa ?")) 
    print("--------\n")

Jülich is a research and development centre in Germany, and it's also known as the Jülich Research Centre.
--------

Jülich Research on Petaflop Architectures (Jüropa) was a collaborative research project in Germany, focused on developing high-performance computing (HPC) architectures for scientific simulations.
--------

Jülich Research on Petaflop Architectures is a research project for developing a petaflop-scale supercomputing architecture.
--------

Jülich is a research and development center in Germany, located in the state of North Rhine-Westphalia. It's primarily known for its work in the field of advanced propulsion systems, particularly for future generation of high-speed space vehicles.
--------

Jülich Research on Petaflop Architectures (Jüropa) was a collaborative project on high-performance computing (HPC) research and development, specifically focused on developing petaflop-class computing architectures.
--------

Jülich is a research and development facility for the Eur

In [ ]:
q_a("What is Juropa in the field of cooking ?") # small check for overfitting

'In the field of cooking, Juropa is a type of sous vide machine, which is a sealed container that is submerged in a bath of water.'